In [12]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import time
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import warnings

In [13]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [14]:
filepath = "/content/gdrive/MyDrive/ColabNotebooks/iot23_combined.csv"

In [15]:
df = pd.read_csv(filepath)

In [16]:
del df['Unnamed: 0']

In [17]:
df

,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,proto_icmp,...,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR
0,2.998796,0,0,0.0,3.0,180.0,0.0,0.0,PartOfAHorizontalPortScan,0,...,0,0,0,1,0,0,0,0,0,0
1,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,PartOfAHorizontalPortScan,0,...,0,0,0,1,0,0,0,0,0,0
2,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,PartOfAHorizontalPortScan,0,...,0,0,0,1,0,0,0,0,0,0
3,2.998804,0,0,0.0,3.0,180.0,0.0,0.0,Benign,0,...,0,0,0,1,0,0,0,0,0,0
4,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,Benign,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444669,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,PartOfAHorizontalPortScan,0,...,0,0,0,1,0,0,0,0,0,0
1444670,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,PartOfAHorizontalPortScan,0,...,0,0,0,1,0,0,0,0,0,0
1444671,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,PartOfAHorizontalPortScan,0,...,0,0,0,1,0,0,0,0,0,0
1444672,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,PartOfAHorizontalPortScan,0,...,0,0,0,1,0,0,0,0,0,0


In [18]:
df['label'].value_counts()

PartOfAHorizontalPortScan     825939
Okiru                         262690
Benign                        197809
DDoS                          138777
C&C                            15100
Attack                          3915
C&C-HeartBeat                    349
C&C-FileDownload                  43
C&C-Torii                         30
FileDownload                      13
C&C-HeartBeat-FileDownload         8
C&C-Mirai                          1
Name: label, dtype: int64

In [19]:
warnings.filterwarnings('ignore')

In [20]:
X = df[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']]
Y = df['label']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=10, test_size=0.2)

In [21]:
clf=RandomForestClassifier(n_estimators =100, random_state=10)

In [22]:
param_grid = { 
    'max_features': ['auto', 'sqrt', 'log2',0.2],
    'max_depth' : [5,6,7,8],
    'criterion' :['gini', 'entropy']
}

In [23]:
CV_rfc = GridSearchCV(estimator=clf, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=10),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [5, 6, 7, 8],
                         'max_features': ['auto', 'sqrt', 'log2', 0.2]})

In [24]:
CV_rfc.best_params_

{'criterion': 'entropy', 'max_depth': 8, 'max_features': 'auto'}

In [25]:
clf=RandomForestClassifier(n_estimators =100, random_state=10, criterion = 'entropy', max_depth = 8, max_features = 'auto')

In [26]:
start = time.time()
print('program start...')
print()

clf.fit(X_train,Y_train)
print()

print('prediction:')
y_pred=clf.predict(X_test)
print(y_pred)
print()

print('Score:')
score = clf.score(X_test,Y_test)
print(score)

end = time.time()
print('program end...')
print()
print('time cost: ')
print(end - start, 'seconds')

program start...


prediction:
['DDoS' 'PartOfAHorizontalPortScan' 'PartOfAHorizontalPortScan' ...
 'PartOfAHorizontalPortScan' 'PartOfAHorizontalPortScan' 'Benign']

Score:
0.7296070050357347
program end...

time cost: 
59.03452920913696 seconds


In [27]:
print("Classifiction Report :")
print(classification_report(Y_test, y_pred))

Classifiction Report :
                            precision    recall  f1-score   support

                    Attack       0.99      0.99      0.99       798
                    Benign       0.95      0.56      0.70     39535
                       C&C       1.00      0.10      0.19      3102
          C&C-FileDownload       0.89      0.89      0.89         9
             C&C-HeartBeat       1.00      0.12      0.21        69
C&C-HeartBeat-FileDownload       1.00      0.67      0.80         3
                 C&C-Torii       0.00      0.00      0.00         7
                      DDoS       1.00      0.82      0.90     27856
              FileDownload       0.60      1.00      0.75         3
                     Okiru       0.77      0.00      0.00     52607
 PartOfAHorizontalPortScan       0.68      1.00      0.81    164946

                  accuracy                           0.73    288935
                 macro avg       0.81      0.56      0.57    288935
              weighted 